In [165]:
# JOINS
# LEFT JOIN <==> LEFT OUTER JOIN
# RIGHT JOIN <==> RIGHT OUTER JOIN
# INNER JOIN <==> JOIN
# FULL OUTER JOIN
# LEFT JOIN EXCLUDING INNER JOIN
# RIGHT JOIN EXCLUDING INNER JOIN
# OUTER JOIN EXCLUDING INNER JOIN
# CROSS JOIN == INNER JOIN ???
# LEFT SEMI JOIN <==> 子查询WHERE IN 性能???
# LEFT ANTI JOIN <==> 子查询WHERE NOT IN 性能???
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()
# 测试数据1：学生语文成绩
chinese_score = [("Tom", 79),
                 ("Lily", 84),
                 ("Sophia", 99),
                 ("Jerry", 80),
                 ("Chery", 67)]
chinese_scheme = ['name', 'chinese_score']
df_chinese = spark.createDataFrame(chinese_score, chinese_scheme)

# 测试数据2：学生数学成绩
math_score = [("Tom", 79),
              ("Tom", 80),
              ("Lily", 84),
              ("Jerry", 80),
              ("Chery", 67),
              ("Jack", 77)]
math_schema = ['name', 'math_score']
df_math = spark.createDataFrame(math_score, math_schema)

In [166]:
df_chinese.createOrReplaceTempView("tb_chinese_score")
df_chinese.show()

+------+-------------+
|  name|chinese_score|
+------+-------------+
|   Tom|           79|
|  Lily|           84|
|Sophia|           99|
| Jerry|           80|
| Chery|           67|
+------+-------------+



In [167]:
df_math.createOrReplaceTempView("tb_math_score")
df_math.show()

+-----+----------+
| name|math_score|
+-----+----------+
|  Tom|        79|
|  Tom|        80|
| Lily|        84|
|Jerry|        80|
|Chery|        67|
| Jack|        77|
+-----+----------+



In [168]:
spark.sql("select * from tb_chinese_score left join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name").show()

+------+-------------+-----+----------+
|  name|chinese_score| name|math_score|
+------+-------------+-----+----------+
|   Tom|           79|  Tom|        79|
|   Tom|           79|  Tom|        80|
| Jerry|           80|Jerry|        80|
| Chery|           67|Chery|        67|
|  Lily|           84| Lily|        84|
|Sophia|           99| null|      null|
+------+-------------+-----+----------+



In [169]:
spark.sql("select * from tb_chinese_score right join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name").show()

+-----+-------------+-----+----------+
| name|chinese_score| name|math_score|
+-----+-------------+-----+----------+
| null|         null| Jack|        77|
|  Tom|           79|  Tom|        79|
|  Tom|           79|  Tom|        80|
|Jerry|           80|Jerry|        80|
|Chery|           67|Chery|        67|
| Lily|           84| Lily|        84|
+-----+-------------+-----+----------+



In [170]:
spark.sql("select * from tb_chinese_score join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name").show()

+-----+-------------+-----+----------+
| name|chinese_score| name|math_score|
+-----+-------------+-----+----------+
|  Tom|           79|  Tom|        79|
|  Tom|           79|  Tom|        80|
|Jerry|           80|Jerry|        80|
|Chery|           67|Chery|        67|
| Lily|           84| Lily|        84|
+-----+-------------+-----+----------+



In [171]:
spark.sql("select * from tb_chinese_score inner join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name").show()

+-----+-------------+-----+----------+
| name|chinese_score| name|math_score|
+-----+-------------+-----+----------+
|  Tom|           79|  Tom|        79|
|  Tom|           79|  Tom|        80|
|Jerry|           80|Jerry|        80|
|Chery|           67|Chery|        67|
| Lily|           84| Lily|        84|
+-----+-------------+-----+----------+



In [172]:
spark.sql("select * from tb_chinese_score full outer join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name").show()

+------+-------------+-----+----------+
|  name|chinese_score| name|math_score|
+------+-------------+-----+----------+
|  null|         null| Jack|        77|
|   Tom|           79|  Tom|        79|
|   Tom|           79|  Tom|        80|
| Jerry|           80|Jerry|        80|
| Chery|           67|Chery|        67|
|  Lily|           84| Lily|        84|
|Sophia|           99| null|      null|
+------+-------------+-----+----------+



In [173]:
spark.sql("select * from tb_chinese_score a left join tb_math_score b"
          " on a.name = b.name"
          " where b.name is null ").show()

+------+-------------+----+----------+
|  name|chinese_score|name|math_score|
+------+-------------+----+----------+
|Sophia|           99|null|      null|
+------+-------------+----+----------+



In [174]:
spark.sql("select * from tb_chinese_score right join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name"
          " where tb_chinese_score.name is null ").show()

+----+-------------+----+----------+
|name|chinese_score|name|math_score|
+----+-------------+----+----------+
|null|         null|Jack|        77|
+----+-------------+----+----------+



In [175]:
spark.sql("select * from tb_chinese_score full outer join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name"
          " where tb_chinese_score.name is null or tb_math_score.name is null ").show()

+------+-------------+----+----------+
|  name|chinese_score|name|math_score|
+------+-------------+----+----------+
|  null|         null|Jack|        77|
|Sophia|           99|null|      null|
+------+-------------+----+----------+



In [176]:
spark.sql("select * from tb_chinese_score left outer join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name").show()

+------+-------------+-----+----------+
|  name|chinese_score| name|math_score|
+------+-------------+-----+----------+
|   Tom|           79|  Tom|        79|
|   Tom|           79|  Tom|        80|
| Jerry|           80|Jerry|        80|
| Chery|           67|Chery|        67|
|  Lily|           84| Lily|        84|
|Sophia|           99| null|      null|
+------+-------------+-----+----------+



In [177]:
spark.sql("select * from tb_chinese_score right outer join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name").show()

+-----+-------------+-----+----------+
| name|chinese_score| name|math_score|
+-----+-------------+-----+----------+
| null|         null| Jack|        77|
|  Tom|           79|  Tom|        79|
|  Tom|           79|  Tom|        80|
|Jerry|           80|Jerry|        80|
|Chery|           67|Chery|        67|
| Lily|           84| Lily|        84|
+-----+-------------+-----+----------+



In [178]:
spark.sql("select * from tb_chinese_score cross join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name").show()

+-----+-------------+-----+----------+
| name|chinese_score| name|math_score|
+-----+-------------+-----+----------+
|  Tom|           79|  Tom|        79|
|  Tom|           79|  Tom|        80|
|Jerry|           80|Jerry|        80|
|Chery|           67|Chery|        67|
| Lily|           84| Lily|        84|
+-----+-------------+-----+----------+



In [179]:
spark.sql("select * from tb_chinese_score left semi join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name").show()

+-----+-------------+
| name|chinese_score|
+-----+-------------+
|  Tom|           79|
|Jerry|           80|
|Chery|           67|
| Lily|           84|
+-----+-------------+



In [180]:
spark.sql("select * from tb_chinese_score where "
          "name in (select name from tb_math_score)").show()

+-----+-------------+
| name|chinese_score|
+-----+-------------+
|  Tom|           79|
|Jerry|           80|
|Chery|           67|
| Lily|           84|
+-----+-------------+



In [181]:
spark.sql("select * from tb_chinese_score left anti join tb_math_score"
          " on tb_chinese_score.name = tb_math_score.name").show()

+------+-------------+
|  name|chinese_score|
+------+-------------+
|Sophia|           99|
+------+-------------+



In [182]:
spark.sql("select * from tb_chinese_score where "
          "name not in (select name from tb_math_score)").show()

+------+-------------+
|  name|chinese_score|
+------+-------------+
|Sophia|           99|
+------+-------------+

